In [52]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 读取图片，记录特征和标签
datadir = './image/classification_imgs/train'
label_names = os.listdir(datadir)

# 直接默认就好
sift = cv2.SIFT_create()

# 读取所有图片，记录出现过的特征
datadir = './image/classification_imgs/train'
features = []
for idx, fname in enumerate(label_names):
    for imgname in os.listdir(f'{datadir}/{fname}'):
        img = cv2.imread(f'{datadir}/{fname}/{imgname}', -1)
        now_features = sift.detectAndCompute(img, None)[1]
        features.extend(list(now_features))
features = np.array(features)

In [ ]:
# KMeans 分类，50 个种类，其中 random_state 要保持始终是同一个值
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=50, random_state=10).fit(features)
print('原始有', features.shape)

In [34]:
# 重新遍历训练集，统计在 Kmeans 分类中各个类出现次数（即直方图统计）
datadir = './image/classification_imgs/train'
train_features, train_labels = [], []
for idx, fname in enumerate(label_names):
    for imgname in os.listdir(f'{datadir}/{fname}'):
        img = cv2.imread(f'{datadir}/{fname}/{imgname}', -1)
        now_features = sift.detectAndCompute(img, None)[1]

        # Kmeans 计算本图中所有 SIFT 特征各自归属哪个类
        now_count = np.array([0 for _ in range(50)])
        for k in kmeans.predict(now_features):
            now_count[k] += 1
        # 归一化哦
        now_count = now_count / np.sum(now_count)
        train_features.append(now_count)
        train_labels.append(idx)
train_features = np.array(train_features, dtype=np.float32)
train_labels = np.array(train_labels, dtype=int)

In [97]:
# 训练 SVM，这里面有重要的参数 C 和 gamma，自己尽量不同尝试吧
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_RBF)
svm.setC(100)
svm.setGamma(100)

svm.train(train_features, cv2.ml.ROW_SAMPLE, train_labels)

# SVM 检测
predict_labels = svm.predict(train_features)[1].flatten()
print('Train Wrong rate:', sum(predict_labels != train_labels)/(train_features.shape[0]))

Train Wrong rate: 0.0012391573729863693


In [39]:
# 测试集，同样在每张图片上统计在 Kmeans 分类中各个类出现次数（即直方图统计）
datadir = './image/classification_imgs/test'
test_features, test_labels = [], []
for idx, fname in enumerate(label_names):
    for imgname in os.listdir(f'{datadir}/{fname}'):
        img = cv2.imread(f'{datadir}/{fname}/{imgname}', -1)
        now_features = sift.detectAndCompute(img, None)[1]

        # Kmeans 计算本图中所有 SIFT 特征各自归属哪个类
        now_count = np.array([0 for _ in range(50)])
        for k in kmeans.predict(now_features):
            now_count[k] += 1
        now_count = now_count / np.sum(now_count)
        # 归一化哦
        test_features.append(now_count)
        test_labels.append(idx)
test_features = np.array(test_features, dtype=np.float32)
test_labels = np.array(test_labels, dtype=int)

In [99]:
# SVM 检测
predict_labels = svm.predict(test_features)[1].flatten()
print('Test Wrong rate:', sum(predict_labels != test_labels)/(test_features.shape[0]))

Test Wrong rate: 0.32857142857142857
